<h2>
Stock Market Cash Trigger</br>
by Ian Kaplan</br>
</h2>

<p>
This notebook explores a stock market timing strategy that is described in David
Alan Carter's book <i>Stock Market Cash Trigger</i>.
</p>
<p>
In the <i>Stock Market Cash Trigger</i> the 200-day moving average for the S&P 500
ETF SPY close prices is compared with the current close SPY price. When the
close prices for SPY trends below the 200-day moving average, this is a
signal to move out of equity instruments into bond instruments.
</p>
<p>
The definition of "trending below" the 200-day moving average is when, on the last
day of a month, the current price is below the 200-day average.
</p>
<p>
If bonds are selected, one of three bonds (JNK, TLT or MUB) is chosen.  The
bond chosen is the bond with the highest three month past return (this is
the ETF rotation technique that is explored at length in
<a href="https://github.com/IanLKaplan/twelve_percent/blob/master/twelve_percent.ipynb"><i>The 12% Solution</i></a>)
</p>
<p>
The ETF instruments used in this notebook are the same instruments
used in the book <i>Stock Market Cash Trigger</i>
</p>
<h3>
S&P 500 ETF
</h3>
<ul>
<li>
<p>
SPY: SPDR S&P 500 ETF Trust
</p>
<p>
SPY tracks a market-cap-weighted index of US large- and midcap stocks selected by
the S&P Committee (e.g., S&P 500).
</p>
<p>
Inception date: Jan 22, 1993
</p>
</li>
</ul>

<h3>
Bond ETFs
</h3>
<ul>
<li>
<p>
TLT: iShares 20+ Year Treasury Bond ETF
</p>
<p>
TLT tracks a market-weighted index of debt issued by the US Treasury
with remaining maturities of 20 years or more.
</p>
<p>
Inception date: 7/22/2002
</p>
</li>
<li>
<p>
JNK: SPDR Bloomberg High Yield Bond ETF
</p>
<p>
JNK tracks a market-weighted index of highly liquid, high-yield, US
dollar-denominated corporate bonds.
</p>
<p>
Inception date: 11/28/2007
</p>
</li>
<li>
<p>
MUB: iShares National Muni Bond ETF
</p>
<p>
MUB tracks a market-weighted index of investment-grade debt issued by state
and local governments and agencies. Interest is exempt from US income tax
and from AMT.
</p>
<p>
Inception date: 09/09/2007
</p>
</li>
</ul>

In [ ]:
from datetime import datetime, timedelta
from numpy import sqrt
from tabulate import tabulate
from typing import List, Tuple
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
from pandas.core.indexes.datetimes import DatetimeIndex
from dateutil.relativedelta import relativedelta
import numpy as np
from pathlib import Path
import tempfile

<h2>
Disclaimer
</h2>
<p>
This notebook is not financial advice, investment advice, or tax advice.
The information in this notebook is for informational and recreational purposes only.
Investment products discussed (ETFs, mutual funds, etc.) are for illustrative purposes
only. This is not a recommendation to buy, sell, or otherwise transact in any of the
products mentioned. Do your own due diligence. Past performance does not guarantee
future returns.
</p>